In [2]:
# Allow auto-reload of external modules 
%reload_ext autoreload
%autoreload 2

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import scipy as scp
import scipy.misc

import numpy as np
import tensorflow as tf

from network.fcn_instance import InstanceSegNet

In [57]:
np.array([[[1, 1, 1], [2, 2, 2]],
            [[3, 3, 3], [4, 4, 4]],
            [[5, 5, 5], [6, 6, 6]]])[0,:,:]

array([[1, 1, 1],
       [2, 2, 2]])

In [67]:
y = np.array([[1,2,3],[4,5,6]])
y2 = np.array([[7,8,9],[10,11,12]])
y3 = np.stack((y,y2),axis=2)
np.shape(y3)
print(y3)

[[[ 1  7]
  [ 2  8]
  [ 3  9]]

 [[ 4 10]
  [ 5 11]
  [ 6 12]]]


In [86]:
x = np.random.randint(0,10,(4,4,3))
w = tf.Variable(x)
lis = [0,2]
lis_ = []
xx = tf.split(2, 3, w)
for i in lis:
    t = tf.slice(w, [0, 0, i], [4, 4, 1])
    lis_.append(t)

new = tf.concat(2,lis_)
s = tf.shape(new)[0]
init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    sp = sess.run(xx)
    m = sess.run(new)

In [93]:
print(x[:,:,0])
print(sp[0][:,:,0])

[[7 2 8 7]
 [0 5 3 9]
 [7 9 4 8]
 [2 7 2 0]]
[[7 2 8 7]
 [0 5 3 9]
 [7 9 4 8]
 [2 7 2 0]]


In [94]:
data_dict = dt.load_weight('data/vgg16.npy')

Successfully load weight file from /Users/jennyzhou/GitLocalRepository/APC_InstanceSegmentation/core/data/vgg16.npy.


In [97]:
for x in data_dict:
    print(data_dict[x][0].shape, data_dict[x][1].shape)

(3, 3, 512, 512) (512,)
(25088, 4096) (4096,)
(3, 3, 512, 512) (512,)
(4096, 4096) (4096,)
(4096, 1000) (1000,)
(3, 3, 512, 512) (512,)
(3, 3, 256, 512) (512,)
(3, 3, 512, 512) (512,)
(3, 3, 512, 512) (512,)
(3, 3, 256, 256) (256,)
(3, 3, 256, 256) (256,)
(3, 3, 128, 256) (256,)
(3, 3, 3, 64) (64,)
(3, 3, 64, 64) (64,)
(3, 3, 128, 128) (128,)
(3, 3, 64, 128) (128,)
